<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_opsmps3copy.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# My App: opensees-mp-s3-copy  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. 

This is a practical walkthrough for defining and deploying an **HPC** app using the Tapis v3 API.

In this notebook we will replicate the opensees-mp-s3 app verbatim

## ✅ Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |



In [1]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


In [2]:
import json

In [3]:
# Make these True once you are done validating
do_makeAppFiles = True
do_upload = True
do_register = True
do_submitJob = True

---
## Connect to Tapis

In [4]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-21T21:43:46+00:00
 Token expires in: 3:55:52.145519
-- LOG IN SUCCESSFUL! --


---
## Configure App

In [5]:
app_id = 'opensees-mp-s3-copy-mine'
# app_version = '0.0.3'

### Autoincrement version

In [6]:
if do_makeAppFiles or do_upload or do_register:
    app_version = OpsUtils.increment_tapis_app_version(t,app_id)

app exists, now latest_app_version 0.0.10
now app_version 0.0.11


### Define folder structure: app_id/version

In [7]:
app_folderName = f'~/MyData/myAuthoredTapisApps/{app_id}/{app_version}'; # your choice

In [8]:
app_folder = os.path.abspath(os.path.expanduser(app_folderName))
os.makedirs(app_folder, exist_ok=True)
print(f'app_folder: {app_folder}\n exists:',os.path.exists(app_folder))

app_folder: /home/jupyter/MyData/myAuthoredTapisApps/opensees-mp-s3-copy-mine/0.0.11
 exists: True


---
## Create the App Files

A Tapis app needs the following **core files**:

We will be creating an app that runs an OpenSeesMP simulation on Stampede3. This app is an identical copy to **opensees-mp-s3** on DesignSafe --
[Template in GitHub](https://github.com/TACC/WMA-Tapis-Templates/tree/main/applications/opensees-mp/opensees-mp-s3)

* **Readme.MD** – App Description (OPTIONAL)<br>
This file is helpful in communicating content to the app user.  

* **app.json** – App Definition<br>
Defines the app’s metadata, inputs, parameters, and execution configuration.    

* **profile.json** – Environment Setup *(optional but common)*<br>
Loads modules or sets environment variables on the execution system. This file will load modules before your script runs. This is executed on the compute node.

* **tapisjob_app.sh** – Wrapper Script<br>
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [9]:
# let's collect the list of files we are creating as we will need this list in upload
appFilesList = [];

#### **Readme.MD** – App Description
This file is helpful in communicating content to the app user.  

In [10]:
thisFilename = 'ReadMe.MD'
if do_makeAppFiles:
    thisText = """\
    # OpenSees-MP Stampede3 version Latest -- copy Silvia
    
    This app uses opensees tacc module with tapis zip runtime.
    Just testing the original app with no modifications
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

appFilesList.append(thisFilename)

#### **app.json** – App Definition
Defines the app’s metadata, inputs, parameters, and execution configuration.    

In [11]:
thisFilename = 'app.json'
if do_makeAppFiles:
    thisText = """\
    {
      "id": "__app_id__",
      "version": "__app_version__",
      "description": "Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.",
      "owner": "${apiUserId}",
      "enabled": true,
      "runtime": "ZIP",
      "runtimeVersion": null,
      "runtimeOptions": null,
      "containerImage": "tapis://cloud.data/corral/tacc/aci/CEP/applications/v3/opensees/latest/OpenSees/opensees.zip",
      "jobType": "BATCH",
      "maxJobs": -1,
      "maxJobsPerUser": -1,
      "strictFileInputs": true,
      "jobAttributes": {
        "execSystemConstraints": null,
        "execSystemId": "stampede3",
        "execSystemExecDir": "${JobWorkingDir}",
        "execSystemInputDir": "${JobWorkingDir}",
        "execSystemOutputDir": "${JobWorkingDir}",
        "execSystemLogicalQueue": "skx",
        "archiveSystemId": "stampede3",
        "archiveSystemDir": "HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
        "archiveOnAppError": true,
        "isMpi": false,
        "mpiCmd": null,
        "parameterSet": {
          "appArgs": [
            {
              "name": "Main Program",
              "arg": "OpenSeesMP",
              "inputMode": "FIXED",
              "notes": {
                "isHidden": true
              }
            },
            {
              "name": "Main Script",
              "description": "The filename only of the OpenSees TCL script to execute. This file should reside in the Input Directory specified. To use with test input, use 'freeFieldEffective.tcl'",
              "arg": null,
              "inputMode": "REQUIRED",
              "notes": {
                "inputType": "fileInput"
              }
            }
          ],
          "containerArgs": [],
          "schedulerOptions": [
            {
              "name": "OpenSees TACC Scheduler Profile",
              "description": "Scheduler profile for the default version of OpenSees",
              "inputMode": "FIXED",
              "arg": "--tapis-profile OpenSees_default",
              "notes": {
                "isHidden": true
              }
            }
          ],
          "envVariables": [],
          "archiveFilter": {
            "includes": [],
            "excludes": [],
            "includeLaunchFiles": true
          }
        },
        "fileInputs": [
          {
            "name": "Input Directory",
            "inputMode": "REQUIRED",
            "sourceUrl": null,
            "targetPath": "inputDirectory",
            "envKey": "inputDirectory",
            "description": "Input directory that includes the tcl script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP",
            "notes": {
              "selectionMode": "directory"
            }
          }
        ],
        "fileInputArrays": [],
        "nodeCount": 2,
        "coresPerNode": 48,
        "memoryMB": 192000,
        "maxMinutes": 120,
        "subscriptions": [],
        "tags": []
      },
      "tags": [
        "portalName: DesignSafe",
        "portalName: CEP"
      ],
      "notes": {
        "label": "OpenSeesMP",
        "helpUrl": "https://www.designsafe-ci.org/user-guide/tools/simulation/#opensees-user-guide",
        "hideNodeCountAndCoresPerNode": false,
        "isInteractive": false,
        "icon": "OpenSees",
        "category": "Simulation"
      }
    }
    """
    thisText = thisText.replace("__app_id__", app_id)
    thisText = thisText.replace("__app_version__", app_version)
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

appFilesList.append(thisFilename)

#### **profile.json** – Environment Setup
This file will load modules before your script runs. This is executed on the compute node.

In [12]:
thisFilename = 'profile.json'
if do_makeAppFiles:
    thisText = """\
    {
        "name": "OpenSees_default",
        "description": "Modules to load for the default version of OpenSees",
        "moduleLoads": [
            {
                "modulesToLoad": [
                    "hdf5/1.14.4",
                    "opensees"
                ],
                "moduleLoadCommand": "module load"
            }
        ],
        "hiddenOptions": [
            "MEM"
        ]
    }
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

appFilesList.append(thisFilename)

#### **tapisjob_app.sh** – Wrapper Script
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [13]:
thisFilename = 'tapisjob_app.sh'
if do_makeAppFiles:
    thisText = """\
    set -x
    
    BINARYNAME=$1
    INPUTSCRIPT=$2
    echo "INPUTSCRIPT is $INPUTSCRIPT"
    
    TCLSCRIPT="${INPUTSCRIPT##*/}"
    echo "TCLSCRIPT is $TCLSCRIPT"
    
    cd "${inputDirectory}"
    
    echo "Running $BINARYNAME"
    
    ibrun $BINARYNAME $TCLSCRIPT
    if [ ! $? ]; then
          echo "OpenSees exited with an error status. $?" >&2
          exit
    fi
    
    cd ..
    """

    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

appFilesList.append(thisFilename)

#### File Check
Look at the files we have written and check for typos or formatting errors.

In [14]:
print(appFilesList)
OpsUtils.show_text_file_in_accordion(app_folder, appFilesList)

['ReadMe.MD', 'app.json', 'profile.json', 'tapisjob_app.sh']


---
## Validate App Locally

In [15]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['validate_app_folder.py'])

In [16]:
# Run validation
validation = OpsUtils.validate_app_folder(app_folder,appFilesList)
if not validation:
    print('Validation Failed: stopping here!!!!')
    a = 3/0

🔍 Validating app folder: /home/jupyter/MyData/myAuthoredTapisApps/opensees-mp-s3-copy-mine/0.0.11

✅ All required files are present.

📄 App ID: opensees-mp-s3-copy-mine
📄 App Name: OpenSees_default
📄 Version: 0.0.11
🔧 Parameters: []
📦 Inputs: []
📤 Outputs: []

App Keys: ['id', 'version', 'description', 'owner', 'enabled', 'runtime', 'runtimeVersion', 'runtimeOptions', 'containerImage', 'jobType', 'maxJobs', 'maxJobsPerUser', 'strictFileInputs', 'jobAttributes', 'tags', 'notes', 'name', 'moduleLoads', 'hiddenOptions']

✅ Basic validation complete. App folder looks good!


---
## Deploy the App

### Get username

In [17]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_username.py'])

In [18]:
username = OpsUtils.get_tapis_username(t)
print('username',username)

username silvia


### Make the directory inside your 'MyData' on designsafe.storage.default.
The apps in this folder are the ones that area actually uploaded.

In [19]:
if do_upload:
    app_system_id="designsafe.storage.default"
    app_path = f"{username}/apps/{app_id}/{app_version}"
    t.files.mkdir(systemId=app_system_id, path=app_path)
    print('app_path',app_path)

app_path silvia/apps/opensees-mp-s3-copy-mine/0.0.11


### Upload files to your deployment system (e.g., DesignSafe default storage)
Using Tapipy (Python SDK) in a Jupyter Notebook

In [20]:
if do_upload:
    for fname in appFilesList:
        fpath = f'{app_folder}/{fname}'
        t.upload(source_file_path=fpath,
                 system_id=app_system_id,
                 dest_file_path=f'{app_path}/{fname}')

#### Verify upload

In [21]:
if do_upload:
    print('app_system_id:',app_system_id)
    print('app_path:',app_path)
    t.files.listFiles(systemId=app_system_id, path=app_path)

app_system_id: designsafe.storage.default
app_path: silvia/apps/opensees-mp-s3-copy-mine/0.0.11


---
## Register the App
This creates the actual App record that Jobs can run.

Do this Using Tapipy (Python)

In [22]:
if do_register:
    # Create (or create a new version) of the app
    with open(f'{app_folder}/app.json') as f:
        app_def = json.load(f)
    t.apps.createAppVersion(**app_def)

### Check that app is up

#### List all apps

In [23]:
if do_register:
    listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
    select = 'id,created,description,version,owner' # Attributes to return in each result.
    orderBy = 'created(asc)'
    results = t.apps.getApps( orderBy=orderBy,
                             select=select)  
    for thisRes in results:
        print('--')
        print(thisRes)

--

created: 2024-12-13T20:15:33.014747Z
description: Run shell commands on remote systems
id: shell-runner-1.0.0
owner: silvia
version: 1.0.0
--

created: 2024-12-14T15:49:21.510206Z
description: Run shell commands on remote systems
id: shell-runner
owner: silvia
version: 1.0.0
--

created: 2025-08-16T16:51:35.355715Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia
owner: silvia
version: latest
--

created: 2025-08-16T18:53:58.422294Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia-new
owner: silvia
version: latest
--

created: 2025-08-17T22:34:01.193589Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-copy-silvia
owner: silvia
vers

#### List the new app's schema

In [24]:
if do_register:
    appMetaData = t.apps.getAppLatestVersion(appId=app_def["id"])
    OpsUtils.display_tapis_app_schema(appMetaData)

########################################
########### TAPIS-APP SCHEMA ###########
########################################
######## appID: opensees-mp-s3-copy-mine
######## version: 0.0.11
########################################
{
  sharedAppCtx: "silvia"
  isPublic: False
  tenant: "designsafe"
  id: "opensees-mp-s3-copy-mine"
  version: "0.0.11"
  description: "Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts."
  owner: "silvia"
  enabled: True
  versionEnabled: True
  locked: False
  runtime: "ZIP"
  runtimeVersion: None
  runtimeOptions: None
  containerImage: "tapis://cloud.data/corral/tacc/aci/CEP/applications/v3/opensees/latest/OpenSees/opensees.zip"
  jobType: "BATCH"
  maxJobs: 2147483647
  maxJobsPerUser: 2147483647
  strictFileInputs: False
  uuid: "50774d66-ee05-4ed7-845e-6bd9c968ab95"
  deleted: False
  created: "2025-08-21T17:47:46.334320Z"
  updated: "2025-08-21T17:47:46.334320Z"
  sharedWi

## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

In [25]:
print('app_id:',app_id)
# print('app_version:',app_version)

app_id: opensees-mp-s3-copy-mine


### App-Specific Input

In [26]:
# initalize
tapisInputAll = {}
tapisInputAll["name"] = 'SilviaApp_OpenSeesMPs3Copy'

tapisInputAll["appId"] = app_id # options: "opensees-express", "opensees-mp-s3", "opensees-2p-s3"
# tapisInputAll["appVersion"] = app_version # always use latest in this Notebook Template
tapisInputAll["maxMinutes"] = 6

# OpenSees-mp-s3 and OpenSees-xp-s3 only:
tapisInputAll["execSystemId"] = "stampede3" # the app runs on stampede only
tapisInputAll["execSystemLogicalQueue"] = "skx-dev" # "skx", "skx-dev"
tapisInputAll["nodeCount"] = 1 # limits set by which compute nodes you use
tapisInputAll["coresPerNode"] = 48 # limits set by which compute nodes you use
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='MyData' # Options: MyData or Work

### Job-Specific Input

In [27]:
# initalize
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'

tapisInput["name"] = tapisInput["name"] + '_' + tapisInput['storage_system']

tapisInput['Main Script'] = 'Ex1a_verymany.Canti2D.Push.mp.tcl'

### Submit

In [28]:
# -----------------------------------------------------
if do_submitJob:
    jobReturns = OpsUtils.run_tapis_job(t,tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    print('JobStatusData:',jobReturns['JobStatusData'])
# -----------------------------------------------------

Creating job_description
job_description {'name': 'SilviaApp_OpenSeesMPs3Copy_CommunityData', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'opensees-mp-s3-copy-mine', 'appVersion': '0.0.11', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Script', 'arg': 'Ex1a_verymany.Canti2D.Push.mp.tcl'}], 'envVariables': [], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'designsafe.storage.default', 'archiveSystemDir': '${EffectiveUserId}/tapis-jobs-archive/${JobCreateDate}/${JobUUID}'}
Submitting Job
Job submitted! ID: 604177ba-8a06-4e88-aa7b-964236d5b3c2-007
job_start_time: 1755798482.5976317

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.16 sec	 Current Status

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (604177ba-8a06-4e88-aa7b-964236d5b3c2-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (604177ba-8a06-4e88-aa7b-964236d5b3c…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (604177ba-8a06-4e88-aa7b-964236d…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (604177ba-8a06-4e88-aa7b-964236d5b3c…

UnboundLocalError: cannot access local variable 'Nfiles' where it is not associated with a value